In [1]:
%matplotlib widget
# import matplotlib
# matplotlib.use("TkAgg")
import tqdm
import glob
from PIL import Image
import numpy as np
import itertools
import matplotlib.pyplot as plt
# noinspection PyUnresolvedReferences
import japanize_matplotlib
import time
# noinspection PyPackageRequirements
import umap
import warnings
warnings.filterwarnings('ignore')

In [2]:
class UmapClass:
    dir_labels = None   # サブフォルダ分類
    png_labels = None   # 画像ラベル
    png_datum = None    # 画像データ配列
    result3d = None     # UMAP配列

    def __init__(self, labels):
        self.dir_labels = labels
        return

    def load(self, path):
        self.png_labels = np.empty(0)
        self.png_datum = np.empty((0,701,701))
        for label in self.dir_labels:
            data = self._load_png(f'{path}/{label}/*.png')
            self.png_datum = np.concatenate((self.png_datum, data))
            self.png_labels = np.concatenate((self.png_labels, np.full(data.shape[0], label)))
        return

    @staticmethod
    def _load_png(path):
        """
        指定されたフォルダ内のグレースケールpngファイルを読み込む
        :param path: glob文字列
        """
        files = glob.glob(path)
        file_sample = Image.open(files[0]).convert('L')
        result_array = np.empty((len(files), file_sample.width, file_sample.height))
        count = 0
        pbar = tqdm.tqdm(files)
        pbar.set_description(path)
        for file in pbar:
            result_array[count] = np.array(Image.open(file).convert('L')).astype('float32') / 255
            count += 1
        return result_array

    def show_sample(self):
        """
        複数のpngデータのプレビュー
        """
        images = {label:self.png_datum[self.png_labels == label][0] for label in self.dir_labels}
        num = len(images)
        fig = plt.figure(figsize=(3 * num, 3))
        count = 1
        for title, image in images.items():
            axis = fig.add_subplot(1, num, count)
            plt.title(title)
            axis.imshow(image, cmap='gray', vmin=0, vmax=1)
            count += 1
        plt.show(block=False)
        return

    def fit(self):
        """
        UMAP学習
        :return: 経過秒数
        """
        t1 = time.time()
        self.result3d = umap.UMAP(n_neighbors=5, n_components=3).fit(
            np.reshape(self.png_datum, (self.png_datum.shape[0], -1))
        )
        return time.time() - t1

    def show_result(self):
        """
        UMAP結果を3D散布図で表示する
        """
        x = self.result3d.embedding_[:, 0]
        y = self.result3d.embedding_[:, 1]
        z = self.result3d.embedding_[:, 2]
        color_cycle= itertools.cycle(["red","green","blue","orange","grey","brown","pink","yellow"])
        fig = plt.figure(figsize=(9, 9))
        ax = fig.add_subplot(111, projection='3d')
        for label in np.unique(self.png_labels):
            ax.scatter(
                x[self.png_labels == label],
                y[self.png_labels == label],
                z[self.png_labels == label],
                c=next(color_cycle), label=label
            )
        ax.legend(loc='upper left')
        plt.show(block=False)

In [3]:
umap_class = UmapClass(['MP掌側', 'MP背側', '手関節背側'])
umap_class.load('data/2021_02_21 US 4クラス')

data/2021_02_21 US 4クラス/手関節背側/*.png: 100%|##########| 300/300 [00:04<00:00, 66.24it/s]


In [4]:
umap_class.show_sample()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
elaps = umap_class.fit()
print(f"経過時間:{elaps}")

経過時間:176.11667799949646


In [6]:
umap_class.show_result()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …